In [1]:
import os, sys
from os import path
sys.path.append(os.path.abspath(path.join(os.getcwd(), os.pardir, 'src')))

import project_config as pc

from data import utils as data_utils
from models.listing_embedding import ListingEmbedder

import re
import numpy as np
import pandas as pd

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = data_utils.load_nyc_listings()

In [3]:
listing_embedder = ListingEmbedder(device='cuda:0')

In [4]:
# embeddings = listing_embedder.from_dataframe(df[:500], 256)

In [5]:
embeddings.shape

NameError: name 'embeddings' is not defined

In [4]:
a = df[df.bedrooms.eq(0)]

In [4]:
def wrapper(row):
    return ListingEmbedder.construct_info_summary(
                room_type=row.room_type,
                neighbourhood_group_cleansed=row.neighbourhood_group_cleansed,
                price=row.price,
                bedrooms=None if pd.isna(row.bedrooms) else row.bedrooms,
                beds=None if pd.isna(row.beds) else row.beds,
                bathrooms_text=None if pd.isna(row.bathrooms_text) else row.bathrooms_text,
                accommodates=None if pd.isna(row.accommodates) else row.accommodates,
            )

In [5]:
wrapper(df.iloc[512])

'Entire home/apt in Brooklyn. 1 bedroom with 4 beds and 1 bath. Accommodates up to 4 people. Costs $105.00 per night'

In [6]:
df.iloc[512]

id                                                                         433914
listing_url                                   https://www.airbnb.com/rooms/433914
room_type                                                         Entire home/apt
neighbourhood_group_cleansed                                             Brooklyn
neighbourhood_cleansed                                              East New York
bathrooms_text                                                             1 bath
bedrooms                                                                        1
beds                                                                            4
accommodates                                                                    4
price                                                                       105.0
latitude                                                                40.673219
longitude                                                              -73.881086
property_type   

In [6]:
df = df[:500]

In [7]:
embeddings

array([[-0.01122258,  0.02900603, -0.02508146, ..., -0.02249013,
        -0.02627785, -0.00582862],
       [-0.01214507,  0.00815747, -0.02764041, ..., -0.0234624 ,
        -0.01677772, -0.00846394],
       [ 0.00322683,  0.0168884 , -0.01908308, ..., -0.01535253,
        -0.03102859, -0.01566676],
       ...,
       [ 0.01111768,  0.01973216, -0.02297438, ..., -0.01283255,
        -0.03752194, -0.00829629],
       [-0.00906407,  0.01199439, -0.02374826, ..., -0.02403551,
        -0.02121878, -0.00759838],
       [-0.00164484,  0.01096731, -0.01210253, ..., -0.01920447,
        -0.02173105, -0.01376694]], dtype=float32)

In [9]:
scores = embeddings @ embeddings.T

In [73]:
df.iloc[0]

id                                                                           2595
listing_url                                     https://www.airbnb.com/rooms/2595
room_type                                                         Entire home/apt
neighbourhood_group_cleansed                                            Manhattan
neighbourhood_cleansed                                                    Midtown
bathrooms_text                                                             1 bath
bedrooms                                                                     <NA>
beds                                                                            1
accommodates                                                                    1
price                                                                       240.0
latitude                                                                 40.75356
longitude                                                               -73.98559
property_type   

In [23]:
df.iloc[163]

id                                                                          82638
listing_url                                    https://www.airbnb.com/rooms/82638
room_type                                                         Entire home/apt
neighbourhood_group_cleansed                                            Manhattan
neighbourhood_cleansed                                               East Village
bathrooms_text                                                             1 bath
bedrooms                                                                        1
beds                                                                            1
accommodates                                                                    2
price                                                                       150.0
latitude                                                                 40.72408
longitude                                                               -73.98154
property_type   

In [24]:
df.filters

AttributeError: 'DataFrame' object has no attribute 'filters'

In [42]:
def filter_by_price(df):
    log_prices = np.log10(df.price.to_numpy())
    return np.abs(log_prices[:, None] - log_prices) <= 0.3

In [50]:
filter_by_price(df)

array([[ True, False, False, ..., False,  True, False],
       [False,  True, False, ..., False, False,  True],
       [False, False,  True, ...,  True, False,  True],
       ...,
       [False, False,  True, ...,  True, False,  True],
       [ True, False, False, ..., False,  True, False],
       [False,  True,  True, ...,  True, False,  True]])

In [61]:
def filter_by_neighborhood(df):
    neighbourhoods = df.neighbourhood_cleansed.to_numpy()
    return neighbourhoods[:, None] != neighbourhoods

In [68]:
filter_by_neighborhood(df)

array([[False,  True,  True, ...,  True,  True,  True],
       [ True, False,  True, ...,  True,  True,  True],
       [ True,  True, False, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ..., False,  True,  True],
       [ True,  True,  True, ...,  True, False,  True],
       [ True,  True,  True, ...,  True,  True, False]])

In [69]:
def filter_by_rooms(df):
    n_berdrooms = df.bedrooms.to_numpy()
    return n_berdrooms

In [71]:
filter_by_rooms(df)[0]

<NA>